---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-data-analysis/resources/0dhYG) course resource._

---

In [ ]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind

# Assignment 4 - Hypothesis Testing
This assignment requires more individual learning than previous assignments - you are encouraged to check out the [pandas documentation](http://pandas.pydata.org/pandas-docs/stable/) to find functions or methods you might not have used yet, or ask questions on [Stack Overflow](http://stackoverflow.com/) and tag them as pandas and python related. And of course, the discussion forums are open for interaction with your peers and the course staff.

Definitions:
* A _quarter_ is a specific three month period, Q1 is January through March, Q2 is April through June, Q3 is July through September, Q4 is October through December.
* A _recession_ is defined as starting with two consecutive quarters of GDP decline, and ending with two consecutive quarters of GDP growth.
* A _recession bottom_ is the quarter within a recession which had the lowest GDP.
* A _university town_ is a city which has a high percentage of university students compared to the total population of the city.

**Hypothesis**: University towns have their mean housing prices less effected by recessions. Run a t-test to compare the ratio of the mean price of houses in university towns the quarter before the recession starts compared to the recession bottom. (`price_ratio=quarter_before_recession/recession_bottom`)

The following data files are available for this assignment:
* From the [Zillow research data site](http://www.zillow.com/research/data/) there is housing data for the United States. In particular the datafile for [all homes at a city level](http://files.zillowstatic.com/research/public/City/City_Zhvi_AllHomes.csv), ```City_Zhvi_AllHomes.csv```, has median home sale prices at a fine grained level.
* From the Wikipedia page on college towns is a list of [university towns in the United States](https://en.wikipedia.org/wiki/List_of_college_towns#College_towns_in_the_United_States) which has been copy and pasted into the file ```university_towns.txt```.
* From Bureau of Economic Analysis, US Department of Commerce, the [GDP over time](http://www.bea.gov/national/index.htm#gdp) of the United States in current dollars (use the chained value in 2009 dollars), in quarterly intervals, in the file ```gdplev.xls```. For this assignment, only look at GDP data from the first quarter of 2000 onward.

Each function in this assignment below is worth 10%, with the exception of ```run_ttest()```, which is worth 50%.

In [ ]:
# Use this dictionary to map state names to two letter acronyms
states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}

In [ ]:
def get_list_of_university_towns():
    '''Returns a DataFrame of towns and the states they are in from the
    university_towns.txt list. The format of the DataFrame should be:
    DataFrame( [ ["Michigan", "Ann Arbor"], ["Michigan", "Yipsilanti"] ],
    columns=["State", "RegionName"]  )

    The following cleaning needs to be done:

    1. For "State", removing characters from "[" to the end.
    2. For "RegionName", when applicable, removing every character from " (" to the end.
    3. Depending on how you read the data, you may need to remove newline character '\n'. '''

    df_univ_towns = pd.read_csv('university_towns.txt', sep='\n', header=None)
    df_univ_towns['raw'] = df_univ_towns.iloc[:, 0]
    df_univ_towns['State'] = df_univ_towns['raw']
    df_univ_towns['RegionName'] = df_univ_towns['State']

    #
    mask_state = df_univ_towns['State'].str.contains('edit')
    mask_region = (mask_state == False)  # reverse boolean

    # State
    df_univ_towns['State'] = (df_univ_towns['State']
                              .str.replace('\[edit\]', '')
                              .str.strip() * mask_state)
    df_univ_towns['State'][mask_region] = np.nan
    df_univ_towns['State'].fillna(method='ffill', inplace=True)

    # Region
    df_univ_towns['RegionName'] = (df_univ_towns['RegionName']
                                   .str.replace(r'\(.*', '')
                                   .str.strip() * mask_region)
    df_univ_towns = df_univ_towns[mask_region]
    df_univ_towns = df_univ_towns[df_univ_towns['RegionName'] != '']
    df_univ_towns = df_univ_towns[['State', 'RegionName']]
    return df_univ_towns

In [ ]:
def get_recession_start():
    '''Returns the year and quarter of the recession start time as a
    string value in a format such as 2005q3'''

    df_gdplev = pd.read_excel('gdplev.xls', skiprows=5)
    df_gdplev_annual = df_gdplev.iloc[3:89,
                       :3]  # annual values are the average quarter values / this df can be neglected
    df_gdp = df_gdplev.iloc[212:, [4, 6]]  # start with 2 quarters earlier to use for shift
    df_gdp.columns = ['Quarter', 'Chained GDP']
    df_gdp['Rolling Min'] = df_gdp['Chained GDP'].rolling(window=2).min()
    df_gdp = df_gdp.iloc[2:, :]

    #
    df_gdp['Recession'] = ((df_gdp['Chained GDP'] < df_gdp['Chained GDP'].shift(1)) &
                           (df_gdp['Chained GDP'].shift(1) < df_gdp['Chained GDP'].shift(2)))
    df_gdp['Start'] = (df_gdp['Recession'].shift(1) == False) & (df_gdp['Recession'])
    start_recession = df_gdp[df_gdp['Start']]['Quarter'].values[0]

    return start_recession

In [ ]:
def get_recession_end():
    '''Returns the year and quarter of the recession end time as a
    string value in a format such as 2005q3'''

    df_gdplev = pd.read_excel('gdplev.xls', skiprows=5)
    df_gdplev_annual = df_gdplev.iloc[3:89,
                       :3]  # annual values are the average quarter values / this df can be neglected
    df_gdp = df_gdplev.iloc[212:, [4, 6]]  # start with 2 quarters earlier to use for shift
    df_gdp.columns = ['Quarter', 'Chained GDP']
    df_gdp['Rolling Min'] = df_gdp['Chained GDP'].rolling(window=2).min()
    df_gdp = df_gdp.iloc[2:, :]

    #
    df_gdp['Recession'] = ((df_gdp['Chained GDP'] < df_gdp['Chained GDP'].shift(1)) &
                           (df_gdp['Chained GDP'].shift(1) < df_gdp['Chained GDP'].shift(2)))
    df_gdp['End'] = (df_gdp['Recession'].shift(-1) == False) & (df_gdp['Recession'])
    end_recession = df_gdp[df_gdp['End']]['Quarter'].values[0]

    return end_recession


get_recession_end()

In [ ]:
def get_recession_bottom():
    '''Returns the year and quarter of the recession bottom time as a
    string value in a format such as 2005q3'''

    df_gdplev = pd.read_excel('gdplev.xls', skiprows=5)
    df_gdp = df_gdplev.iloc[212:, [4, 6]]  # start with 2 quarters earlier to use for shift
    df_gdp.columns = ['Quarter', 'Chained GDP']
    df_gdp['Rolling Min'] = df_gdp['Chained GDP'].rolling(window=2).min()
    df_gdp = df_gdp.iloc[2:, :]

    #
    df_gdp['Recession'] = ((df_gdp['Chained GDP'] < df_gdp['Chained GDP'].shift(1)) &
                           (df_gdp['Chained GDP'].shift(1) < df_gdp['Chained GDP'].shift(2)))

    # Bottom:
    df_gdp['Bottom'] = ((df_gdp['Rolling Min'].shift(-1) == df_gdp['Rolling Min']) &
                        (df_gdp['Recession']))
    bottom_recession = df_gdp[df_gdp['Bottom']]['Quarter'].values[0]

    return bottom_recession

In [ ]:
def convert_housing_data_to_quarters():
    '''Converts the housing data to quarters and returns it as mean
    values in a dataframe. This dataframe should be a dataframe with
    columns for 2000q1 through 2016q3, and should have a multi-index
    in the shape of ["State","RegionName"].

    Note: Quarters are defined in the assignment description, they are
    not arbitrary three month periods.

    The resulting dataframe should have 67 columns, and 10,730 rows.
    '''
    df_zhvi = pd.read_csv('City_Zhvi_AllHomes.csv')
    df_zhvi.drop(df_zhvi.columns[0], axis=1, inplace=True)
    df_zhvi.drop(df_zhvi.columns[2:50], axis=1, inplace=True)

    def col_batch(start, seq, step):
        return (seq.iloc[:, pos:pos + step] for pos in range(start, len(seq.columns), step))

    for i, cols in enumerate(col_batch(2, df_zhvi, 3)):
        quarter = '{}q{}'.format(i // 4 + 2000, i % 4 + 1)
        df_zhvi[quarter] = cols.mean(axis=1)

    mask_cols = df_zhvi.columns.str.extract('(.+-.+)', expand=False).fillna(False) == False
    df_zhvi.columns[mask_cols]
    df_zhvi = df_zhvi.iloc[:, mask_cols]

    df_zhvi['State'] =  df_zhvi['State'].replace(states)
    df_zhvi.set_index(['State', 'RegionName'], inplace=True)

    return df_zhvi


In [ ]:
def run_ttest():
    '''First creates new data showing the decline or growth of housing prices
    between the recession start and the recession bottom. Then runs a ttest
    comparing the university town values to the non-university towns values,
    return whether the alternative hypothesis (that the two groups are the same)
    is true or not as well as the p-value of the confidence.

    Return the tuple (different, p, better) where different=True if the t-test is
    True at a p<0.01 (we reject the null hypothesis), or different=False if
    otherwise (we cannot reject the null hypothesis). The variable p should
    be equal to the exact p value returned from scipy.stats.ttest_ind(). The
    value for better should be either "university town" or "non-university town"
    depending on which has a lower mean price ratio (which is equivilent to a
    reduced market loss).'''

    df_housing = convert_housing_data_to_quarters()
    df_univ_towns = get_list_of_university_towns()
    df_univ_towns['University'] = True  # Splitting rule

    # Merge and split df
    df_merge = pd.merge(df_univ_towns, df_housing, how='outer', left_on=['State', 'RegionName'], right_index=True)
    df_univ = df_merge[df_merge['University'] == True]
    df_nonu = df_merge[df_merge['University'] != True]

    df_merge.sort_values(by=['State', 'RegionName'], inplace=True)

    #
    bottom_recession = get_recession_bottom()
    start_recession = get_recession_start()
    last_quarter = df_univ.columns[[ix-1 for ix, c in enumerate(df_univ.columns) if c == start_recession]][0]

    # Ratio distributions
    ratio_univ = df_univ.loc[:, last_quarter] / df_univ.loc[:, bottom_recession]
    df_univ['Ratio'] = ratio_univ
    ratio_nonu = df_nonu.loc[:, last_quarter] / df_nonu.loc[:, bottom_recession]
    df_nonu['Ratio'] = ratio_nonu

    # from scipy.stats import ttest_ind
    p = ttest_ind(ratio_univ, ratio_nonu, nan_policy='omit')[1]
    different = p < 0.01
    better = ["university town", "non-university town"][ratio_univ.mean() > ratio_nonu.mean()]

    return different, p, better


run_ttest()